In [2]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [3]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


In [5]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)), # Resize image to 150 x 150
    transforms.RandomHorizontalFlip(), # Randomly choose to flip horizontally or not
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [6]:
# set variables for path names
train_path=r"C:\Users\selfa\Desktop\github\galaxy-gesture\TEST_DATA\seg_train"
test_path=r"C:\Users\selfa\Desktop\github\galaxy-gesture\TEST_DATA\seg_test"
pred_path=r"C:\Users\selfa\Desktop\github\galaxy-gesture\TEST_DATA\seg_pred"

In [7]:
#Dataloader

#Path for training and testing directory
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [8]:
#categories
root=pathlib.Path(train_path)
# print(root)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [9]:
print(classes)

['A', 'B']


In [10]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [12]:
model=ConvNet(num_classes=2).to(device)

In [13]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [15]:
num_epochs=100

In [16]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [17]:
print(train_count,test_count)

240 72


In [18]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       


Epoch: 0 Train Loss: tensor(17.1272) Train Accuracy: 0.4375 Test Accuracy: 0.5
Epoch: 1 Train Loss: tensor(10.7848) Train Accuracy: 0.5 Test Accuracy: 0.5277777777777778
Epoch: 2 Train Loss: tensor(4.8958) Train Accuracy: 0.5958333333333333 Test Accuracy: 0.4722222222222222
Epoch: 3 Train Loss: tensor(3.0148) Train Accuracy: 0.7291666666666666 Test Accuracy: 0.5
Epoch: 4 Train Loss: tensor(1.6228) Train Accuracy: 0.7375 Test Accuracy: 0.4583333333333333
Epoch: 5 Train Loss: tensor(1.7990) Train Accuracy: 0.7333333333333333 Test Accuracy: 0.4444444444444444
Epoch: 6 Train Loss: tensor(1.0879) Train Accuracy: 0.8083333333333333 Test Accuracy: 0.4861111111111111
Epoch: 7 Train Loss: tensor(1.3372) Train Accuracy: 0.8125 Test Accuracy: 0.4722222222222222
Epoch: 8 Train Loss: tensor(0.8971) Train Accuracy: 0.8541666666666666 Test Accuracy: 0.5277777777777778
Epoch: 9 Train Loss: tensor(0.6121) Train Accuracy: 0.8708333333333333 Test Accuracy: 0.5555555555555556
Epoch: 10 Train Loss: tensor(